# Part1 - Data Wrangling

### Imports

In [1]:
import numpy as np 
import pandas as pd
import os
import seaborn as sns
import sys
import boto
from boto.s3.key import Key
import zipfile
import boto.s3.connection
from os.path import basename
color = sns.color_palette()
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing, neighbors
from sklearn.preprocessing import LabelEncoder
from uszipcode import ZipcodeSearchEngine
import matplotlib.pyplot as plt

### Importing the properties and transaction details

In [2]:
detail_2016 = pd.read_csv('/Users/sonalichaudhari/Desktop/ADS_MidTerm/Data/properties_2016.csv',low_memory=False)
trans_2016 = pd.read_csv('/Users/sonalichaudhari/Desktop/ADS_MidTerm/Data/train_2016.csv',low_memory=False)
details_2017 = pd.read_csv('/Users/sonalichaudhari/Desktop/ADS_MidTerm/Data/properties_2017.csv',low_memory=False)
trans_2017 = pd.read_csv('/Users/sonalichaudhari/Desktop/ADS_MidTerm/Data/train_2017.csv',low_memory=False)

In [21]:
df1=Cleaning(detail_2016,trans_2016)
df2= Cleaning(details_2017,trans_2017)
Merge(df1,df2)

0.590064679007
2016


/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/sonalichaudhari/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


0.574331578471
2017
Combined CSV created successfully!!!


In [13]:
df1.shape

(89611, 26)

In [7]:
def Cleaning(detail,trans):
    # 70% missing data
    missing_data = detail.isnull().sum().to_frame(name='Missing_Count').reset_index()
    missing_data['Missing_percent'] = (missing_data['Missing_Count']/detail.shape[0])*100
    missing_data.head()
    missing_data = missing_data.sort_values(by='Missing_percent', axis=0, ascending=True)
    features = missing_data['index'][(missing_data.Missing_percent <30)]
    detail = detail[features]

    # Separating the year and month in transaction dataframe
    trans['transactiondate'] = pd.to_datetime(trans['transactiondate'])
    trans['year'] = trans['transactiondate'].dt.year
    trans['month'] = trans['transactiondate'].dt.month

    # Merging the properties and transaction data into a single dataframe
    df =   pd.merge(trans,detail,on='parcelid', how ='left')

    # Getting rid of redudndant columns
    del df['censustractandblock']
    del df['fullbathcnt']
    del df['calculatedbathnbr']
    del df['finishedsquarefeet12']

    # Latitude and longitude format 
    df = df.dropna(subset=['latitude'])
    df = df.dropna(subset=['longitude'])
    df['latitude'] = df['latitude'] / 1000000
    df['longitude'] = df['longitude'] / 1000000

    # Zipcode
    z = df[['regionidzip','regionidcounty']][df.regionidcounty ==3101.0]
    search = ZipcodeSearchEngine()
    zipcode = z['regionidzip'][z.regionidzip <= 100000].mode().values[0]
    df.loc[df['regionidzip'] > 100000, 'regionidzip'] = zipcode

    df.loc[df.regionidzip.isnull(), 'regionidzip']=search.by_coordinate(df['latitude'].values[0],df['longitude'].values[0], radius=30, returns=1)[0]['Zipcode']

    # propertycountylandusecode
    df['propertycountylandusecode'] = df['propertycountylandusecode'].apply(lambda x: np.nan if x == 0 else x) 
    df['propertycountylandusecode'] = df['propertycountylandusecode'].fillna(df['propertycountylandusecode'].mode()[0]) 
    lb_make = LabelEncoder()
    df["propertycountylandusecode"] = lb_make.fit_transform(df["propertycountylandusecode"])
    df[["propertycountylandusecode"]]
    lb_makeDesc = LabelEncoder()
    lb_makeDesc = LabelEncoder()
    df["transactiondate"] = lb_makeDesc.fit_transform(df["transactiondate"])

    df['taxvaluedollarcnt'].fillna(df['taxvaluedollarcnt'].median(), inplace=True)
    df['landtaxvaluedollarcnt'].fillna(df['landtaxvaluedollarcnt'].median(), inplace=True)
    df['taxamount'].fillna(df['taxamount'].median(), inplace=True)

    # yearbuilt
    knn_year(df)

    # age
    df['age'] =2017-df['yearbuilt']

    # calculatedfinishedsquarefeet
    df = df[df.calculatedfinishedsquarefeet>=120]
    df['structuretaxvaluedollarcnt'].fillna(df['structuretaxvaluedollarcnt'].median(), inplace=True)

    # lotsizesquarefeet
    print(df.year[0])
    if df.year[0] ==2017:
        out = 2000000
    else:
        out = 3000000
    lot_median =df['lotsizesquarefeet'].median()
    df.loc[df.lotsizesquarefeet > out, 'lotsizesquarefeet']= lot_median
    df['lotsizesquarefeet'].fillna(df['lotsizesquarefeet'].median(), inplace=True)

    # regioncityid
    missingcity(df)
    path = '/Users/sonalichaudhari/Desktop/ADS_MidTerm/Data/New_Data/'
    path = path+'CSV'+str(df.year[0])+'.csv'
    df.to_csv(path, sep=',')
    return df

In [10]:
# path = '/Users/sonalichaudhari/Desktop/ADS_MidTerm/Data/New_Data/'
# path = path+'CSV'+str(df.year[0])+'.csv'
# path

In [8]:
def knn_year(yeardf):
    whole = ['latitude','longitude','yearbuilt','parcelid']
    base = ['latitude','longitude']
    target_column = 'yearbuilt'

    null = yeardf[target_column].isnull()
    not_null = ~null
    num_miss = null.sum()
    if num_miss ==0:
        print('No Null Values!!!')
    else:
        X_train = yeardf.loc[not_null, whole].sample(frac = 1)

        # X_train.latitude.astype(int)
        # X_train.longitude.astype(int)
        # X_train.yearbuilt.astype(int)
        X=X_train[base].values

        y = X_train[target_column].astype(int)
        y = y.values

        Y_target =yeardf.loc[null, whole].sample(frac = 1)
        Y_target.latitude.astype(int)
        Y_target.longitude.astype(int)

        Y =Y_target[['latitude','longitude']].values

        clf = neighbors.KNeighborsClassifier()
        clf.fit(X,y)
        accuracy = clf.score(X, y)
        print(accuracy) 

        #Y = Y.reshape(len(X), -1)
        prediction = clf.predict(Y)
        Y_target.loc[Y_target.yearbuilt.isnull(),'yearbuilt'] = prediction
        yeardf.loc[yeardf.parcelid.isin(Y_target.parcelid), ['yearbuilt']] = Y_target[['yearbuilt']]

In [9]:
def missingcity(m):
    city = m[['regionidcounty','parcelid', 'regionidcity']]
    Mode = m['regionidcity'].mode()[0]
    county= list(city['regionidcounty'].unique())

    for c in county:
        temp = city[city.regionidcounty == c]
        if(temp.shape == temp[temp.isnull().any(axis=1)].shape ):
                temp['regionidcity'].fillna(Mode, inplace=True)
        else:
                c = temp['regionidcity'].mode()[0]
                temp['regionidcity'].fillna(c, inplace=True)
        city.loc[city.parcelid.isin(temp.parcelid), ['regionidcity']] = temp[['regionidcity']]
    city['regionidcity'].fillna(city['regionidcity'].mode()[0], inplace = True)      
    m.loc[m.parcelid.isin(city.parcelid), ['regionidcity']] = city[['regionidcity']]

# Uploading csv to AWS

In [37]:
AWS_ACCESS_KEY_ID=''
AWS_SECRET_ACCESS_KEY=''
try:
    connect = boto.connect_s3(AWS_ACCESS_KEY_ID,AWS_SECRET_ACCESS_KEY)

except:
    print("incorrect aws access key and/or secret key")

In [879]:
#connection = boto.connect_s3(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

#csvFile = 'zillowData.csv'
zipname = "zillowdata.zip"
z = zipfile.ZipFile(zipname, "w")
z.write('zillowData.csv')
cur_path
bucket_name = 'zillowdatampsc'

loc = boto.s3.connection.Location.USWest

bucket = connect.create_bucket(bucket_name, location=loc)
def percent_cb(complete, total):
    sys.stdout.write('.')
    sys.stdout.flush()

s3key = boto.s3.key.Key(bucket)
s3key.key = zipname
s3key.set_contents_from_filename(zipname, cb=percent_cb, num_cb=10)

### Coverting to csv

In [18]:
def Merge(data1, data2):
    df_concat = pd.concat([data1, data2]).reset_index()
    del df_concat['index']
    df_concat.to_csv('/Users/sonalichaudhari/Desktop/ADS_MidTerm/Data/New_Data/Clean_Combined.csv', sep=',')
    print('Combined CSV created successfully!!!')